In [ ]:
import pandas as pd
import pandas as pd
from helper import *
import pickle


brain = pd.read_csv("./data/brain.csv")
main = pd.read_csv("./data/data.csv")


In [ ]:

brain = pd.read_csv("./data/brain.csv")
brain = brain.drop(brain.columns[[0, 1, 2, 3]], 1)
with open('./data/columns/brain_subcor', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(list(brain), filehandle)

with open('./data/columns/brain_subcor', 'rb') as filehandle:
    # read the data as binary data stream
    brainCol = pickle.load(filehandle)

assert len(brain[brainCol].columns) < len(pd.read_csv("./data/brain.csv").columns) 
#len(brain[brainCol].columns)


In [ ]:
import pickle

neighbor = ['su_risk_p_1', 
'su_risk_p_2_3',
'su_risk_p_4_5', 
'neighborhood1_2_3_p',
'neighborhood_crime_y']

with open('./data/columns/neighbor', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(neighbor, filehandle)

In [ ]:
family = ['fes_youth',
'fes_p_ss_fc_pr',
'macv_p_ss_fs',
'macv_p_ss_fo',
'macv_p_ss_isr',
'macv_p_ss_fr',
'macv_p_ss_r',
'demo_prnt_age_v2',
'demo_prnt_marital_v2',
'demo_comb_income_v2',
'demo_fam_exp',
'demo_yrs_1',
'demo_yrs_2',
'parent_rules_q1',
'parent_rules_q4',
'parent_rules_q7']

with open('./data/columns/family', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(family, filehandle)

In [ ]:
parent = [
'asr_scr_perstr_t',
'asr_scr_anxdep_t',
'asr_scr_withdrawn_t',
'asr_scr_somatic_t',
'asr_scr_thought_t',
'asr_scr_attention_t',
'asr_scr_aggressive_t',
'asr_scr_rulebreak_t',
'asr_scr_intrusive_t',
'asr_scr_internal_t',
'asr_scr_external_t',
'asr_scr_totprob_t',
'asr_scr_depress_t',
'asr_scr_anxdisord_t',
'asr_scr_somaticpr_t',
'asr_scr_avoidant_t',
'asr_scr_adhd_t',
'asr_scr_antisocial_t',
'asr_scr_inattention_t',
'asr_scr_hyperactive_t']

with open('./data/columns/parent', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(parent, filehandle)

In [ ]:
sex = [
'sex']

with open('./data/columns/sex', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(sex, filehandle)

In [ ]:
import rpy2.robjects.packages as rpackages
# import R's utility package
utils = rpackages.importr('utils')
# select a mirror for R packages
utils.chooseCRANmirror(ind=1) 
# R package names
packnames = ('ggplot2', 'hexbin')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))


In [ ]:
import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
r = robjects.r
output = r.source("R_script_run_in_python.R")

In [ ]:
a = df['subjectkey']
b = df['aggressive_sumscore.x']
print(a[0:4])
print(b[0:4])
result = pd.concat([a[0:4], b[0:4]], axis=1)
print(result)

In [ ]:
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

base = importr('base')
ROSE = importr('ROSE')



with open('./data/columns/family', 'rb') as filehandle:
    # read the data as binary data stream
    family = pickle.load(filehandle)

df = pd.read_csv("./data/aggr-all.csv")
df = pd.concat([df[family] , df['y']], axis=1)
# result.to_csv("./data/temp/temp.csv")
with localconverter(ro.default_converter + pandas2ri.converter):
  r_from_pd_df = ro.conversion.py2rpy(df)

seed=3
N=1000
robjects.globalenv["df"] = r_from_pd_df
robjects.r(f'''
          data.rose <- ROSE(y~., data=df, seed={seed},
                  N={N})$data
''')

with localconverter(ro.default_converter + pandas2ri.converter):
  pd_from_r_df = ro.conversion.rpy2py(robjects.globalenv["data.rose"] )

pd_from_r_df


In [ ]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import pandas as pd
import pickle
base = importr('base')
ROSE = importr('ROSE')


def preprosessing(columns, seed=3, size=2000, original_file="aggr-all.csv", original=False):
    df = pd.read_csv(f"./data/{original_file}")

    for index, item in enumerate(columns):

        with open(f'./data/columns/{item}', 'rb') as filehandle:
            item = pickle.load(filehandle)
        if(index == 0):
            temp = pd.concat([df[item], df['y']], axis=1)
        else:
            temp = pd.concat([df[item], temp], axis=1)

    if not original:
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_df = ro.conversion.py2rpy(temp)

        ro.globalenv["df"] = r_from_pd_df
        ro.r(f'''
                if('sex' %in% colnames(df))
                df$sex <- ifelse(df$sex == "M" , 1, 0)
            
                data.rose <- ROSE(y~., data=df, seed={seed},
                        N={size})$data
    ''')

        with localconverter(ro.default_converter + pandas2ri.converter):
            pd_from_r_df = ro.conversion.rpy2py(ro.globalenv["data.rose"])

        assert 'y' in pd_from_r_df
        return pd_from_r_df
    else:
        return temp


In [ ]:
# from helper import *
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import (confusion_matrix, 
                           accuracy_score)

col = ["family", "sex", "neighbor", "brain_subcor"]
df = preprosessing(col, size=8000)

ytrain = df[['y']]
Xtrain = df.drop('y', axis=1)
log_reg = sm.Logit(ytrain, Xtrain).fit()
print(log_reg.summary())

df = preprosessing(col, original=True)
ytest = df[['y']]
Xtest = df.drop('y', axis=1)
yhat = log_reg.predict(Xtest)
prediction = list(map(round, yhat))
 
# confusion matrix
cm = confusion_matrix(ytest, prediction) 
print ("Confusion Matrix : \n", cm) 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(ytest, prediction))


In [ ]:

# family = ['fes_youth',
# 'fes_p_ss_fc_pr',
# 'macv_p_ss_fs',
# 'macv_p_ss_fo',
# 'macv_p_ss_isr',
# 'macv_p_ss_fr',
# 'macv_p_ss_r',
# 'demo_prnt_age_v2',
# 'demo_prnt_marital_v2',
# 'demo_comb_income_v2',
# 'demo_fam_exp',
# 'demo_yrs_1',
# 'demo_yrs_2',
# 'parent_rules_q1',
# 'parent_rules_q4',
# 'parent_rules_q7']


import pandas as pd
cor2cor = pd.read_csv("./data/brain_cor2cor.csv")
cor2subcor = pd.read_csv("./data/brain_cor2subcor.csv")
data = pd.read_csv("./data/data.csv")
with open('./data/columns/cor2cor', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(cor2cor.drop(columns=['subjectkey'], axis=1).columns, filehandle)

with open('./data/columns/cor2subcor', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(cor2subcor.drop(columns=['subjectkey'], axis=1).columns, filehandle)

print(cor2cor.iloc[:, :3].head(2))

print(cor2subcor.iloc[:, :3].head(2))
print(data.iloc[:, :3].head(3))


df1 = pd.merge(data, cor2cor, on='subjectkey', how='inner')
df2 = pd.merge(df1, cor2subcor, on='subjectkey', how='inner')

print(df2.columns)
df2.to_csv('./data/new-aggr-all.csv')

In [7]:
df = pd.read_csv('./data/new-aggr-all-processed.csv')
# df = pd.concat([main[main['aggressive_sumscore']<=-2.0205], main[main['aggressive_sumscore']>=0.2168869]])

# df.to_csv('./data/new-aggr-all-processed.csv')
def f(row):
    if row['aggressive_sumscore'] <= -2.0205:
        val = 0
    else:
        val = 1
    return val

df['y'] =  df.apply(f, axis=1)

df.to_csv("./data/new-aggr-all-processed.csv")

In [10]:
from helper import *
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import (confusion_matrix, 
                           accuracy_score)

col = ["family", "sex", "neighbor", "brain_subcor"]
df = preprosessing(col, size=8000)

df = pd.get_dummies(df, columns=['sex'], drop_first=True)

ytrain = df[['y']]
Xtrain = df.drop('y', axis=1)
log_reg = sm.Logit(ytrain, Xtrain).fit()
print(log_reg.summary())

# df = preprosessing(col, orginal=True)
# ytest = df[['y']]
# Xtest = df.drop('y', axis=1)
# yhat = log_reg.predict(Xtest)
# prediction = list(map(round, yhat))
 
# # confusion matrix
# cm = confusion_matrix(ytest, prediction) 
# print ("Confusion Matrix : \n", cm) 
# # accuracy score of the model
# print('Test accuracy = ', accuracy_score(ytest, prediction))


Optimization terminated successfully.
         Current function value: 0.602545
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7936
Method:                           MLE   Df Model:                           63
Date:                Fri, 10 Jun 2022   Pseudo R-squ.:                  0.1307
Time:                        17:39:52   Log-Likelihood:                -4820.4
converged:                       True   LL-Null:                       -5545.1
Covariance Type:            nonrobust   LLR p-value:                2.076e-261
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
rsfmri_cor_ngd_cerc_scs_thp    -0.3239      0.321     -1.008      0.314   